In [ ]:
from maplib import Mapping, explore
import polars as pl
pl.Config.set_fmt_str_lengths(200)

In [ ]:
m = Mapping()

In [ ]:
publicID = "urn:ukgrid:bridgewater"
m.read_triples("bridgwater/cgmes_206008_bridgwater_sgp_2023_09_14_20.25.21.xml", base_iri=publicID)
m.read_triples("bridgwater/CoreEquipment.nt")
m.read_triples("bridgwater/Operation.nt")

In [ ]:
await explore(m)

## Oppgave 0

In [ ]:
m.query("""
PREFIX cim:     	<http://iec.ch/TC57/2013/CIM-schema-cim16#>
PREFIX rdf: 		<http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs:    	<http://www.w3.org/2000/01/rdf-schema#>
PREFIX lede:    	<https://lede.no/cim/>
SELECT ?instance_name 
WHERE {
    ?instance rdf:type cim:PowerTransformer .
    ?instance cim:IdentifiedObject.name ?instance_name .
} 
""")

## Oppgave 1

In [ ]:
m.query("""
PREFIX cim:     	<http://iec.ch/TC57/2013/CIM-schema-cim16#>
PREFIX rdf: 		<http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs:    	<http://www.w3.org/2000/01/rdf-schema#>
PREFIX lede:    	<https://lede.no/cim/>
SELECT ?instance_name ?end_name ?base_voltage_name ?nominal_voltage 
WHERE {
    ?instance rdf:type cim:PowerTransformer .
    ?instance cim:IdentifiedObject.name ?instance_name .
    ?end cim:PowerTransformerEnd.PowerTransformer ?instance . 
    ?end cim:IdentifiedObject.name ?end_name .
    ?end cim:TransformerEnd.BaseVoltage ?base_voltage .
    ?base_voltage cim:IdentifiedObject.name ?base_voltage_name .
    ?base_voltage cim:BaseVoltage.nominalVoltage ?nominal_voltage .
} 
""")

## Oppgave 2

In [ ]:
m.query("""
PREFIX cim:     	<http://iec.ch/TC57/2013/CIM-schema-cim16#>
PREFIX rdf: 		<http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs:    	<http://www.w3.org/2000/01/rdf-schema#>
PREFIX lede:    	<https://lede.no/cim/>
PREFIX xsd:         <http://www.w3.org/2001/XMLSchema#>
SELECT ?instance_name ?end_name ?base_voltage_name ?nominal_voltage 
WHERE {
    ?instance rdf:type cim:PowerTransformer .
    ?instance cim:IdentifiedObject.name ?instance_name .
    ?end cim:PowerTransformerEnd.PowerTransformer ?instance . 
    ?end cim:IdentifiedObject.name ?end_name .
    ?end cim:TransformerEnd.BaseVoltage ?base_voltage .
    ?base_voltage cim:IdentifiedObject.name ?base_voltage_name .
    ?base_voltage cim:BaseVoltage.nominalVoltage ?nominal_voltage .
    FILTER(xsd:float(?nominal_voltage) > 11)
} 
""")

## Oppgave 3

In [ ]:
df = m.query("""
PREFIX cim:     	<http://iec.ch/TC57/2013/CIM-schema-cim16#>
PREFIX rdf: 		<http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs:    	<http://www.w3.org/2000/01/rdf-schema#>
PREFIX lede:    	<https://lede.no/cim/>
PREFIX xsd:         <http://www.w3.org/2001/XMLSchema#>
SELECT ?instance_name ?end_name ?base_voltage_name ?nominal_voltage 
WHERE {
    ?instance rdf:type cim:PowerTransformer .
    ?instance cim:IdentifiedObject.name ?instance_name .
    ?end cim:PowerTransformerEnd.PowerTransformer ?instance . 
    ?end cim:IdentifiedObject.name ?end_name .
    ?end cim:TransformerEnd.BaseVoltage ?base_voltage .
    ?base_voltage cim:IdentifiedObject.name ?base_voltage_name .
    ?base_voltage cim:BaseVoltage.nominalVoltage ?nominal_voltage .
    FILTER(xsd:float(?nominal_voltage) > 11)
} 
""")
df

In [ ]:
df = df.with_columns(
    pl.col("nominal_voltage").cast(pl.Float64)
)
df

In [ ]:
df = df.sort(pl.col("nominal_voltage"),descending=True)
df

In [ ]:
#await explore(m, popup=False)

## Oppgave 4

In [ ]:
m.query("""
PREFIX cim:     	<http://iec.ch/TC57/2013/CIM-schema-cim16#>
PREFIX rdf: 		<http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs:    	<http://www.w3.org/2000/01/rdf-schema#>
PREFIX lede:    	<https://lede.no/cim/>
PREFIX xsd:         <http://www.w3.org/2001/XMLSchema#>
SELECT ?segment_name ?limit_name ?current_limit_value 
WHERE {
    ?segment a cim:ACLineSegment .
    ?segment cim:IdentifiedObject.name ?segment_name .
    ?limit_set cim:OperationalLimitSet.Equipment ?segment .
    ?limit_set cim:IdentifiedObject.name ?limit_name .
    ?limit_set a cim:OperationalLimitSet .
    ?current_limit a cim:CurrentLimit .
    ?current_limit cim:OperationalLimit.OperationalLimitSet ?limit_set .
    ?current_limit cim:CurrentLimit.value ?current_limit_value . 
} 
""")

## Oppgave 5 

In [ ]:
df = m.query("""
PREFIX cim: <http://iec.ch/TC57/2013/CIM-schema-cim16#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?pt_name ?nv1 ?nv2 WHERE {
    ?pt a cim:PowerTransformer . #?pt er en PowerTransformer
    ?pt cim:IdentifiedObject.name ?pt_name . #Navnet til ?pt er ?pt_name
    # ?pt har tre power transformer end
    ?pte1 a cim:PowerTransformerEnd .
    ?pte2 a cim:PowerTransformerEnd .
    ?pte1 cim:PowerTransformerEnd.PowerTransformer ?pt .
    ?pte2 cim:PowerTransformerEnd.PowerTransformer ?pt .
    # Disse power transformer end har hver sin nominelle spenning ?nv1, ?nv2 og ?nv3
    ?pte1 cim:TransformerEnd.BaseVoltage ?base_voltage1 .
    ?base_voltage1 cim:IdentifiedObject.name ?base_voltage1_name .
    ?base_voltage1 cim:BaseVoltage.nominalVoltage ?nv1 .
    ?pte2 cim:TransformerEnd.BaseVoltage ?base_voltage2 .
    ?base_voltage2 cim:IdentifiedObject.name ?base_voltage2_name .
    ?base_voltage2 cim:BaseVoltage.nominalVoltage ?nv2 .
    FILTER(
        xsd:float(?nv1) > xsd:float(?nv2)
        )
    } LIMIT 100
""")
df

Diagnosespørring for å sjekke antall power transformer end pr. power transformer

In [ ]:
df = m.query("""
PREFIX cim: <http://iec.ch/TC57/2013/CIM-schema-cim16#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?pt_name ?pte_name WHERE {
    ?pt a cim:PowerTransformer .
    ?pt cim:IdentifiedObject.name ?pt_name . 
    ?pte a cim:PowerTransformerEnd .
    ?pte cim:PowerTransformerEnd.PowerTransformer ?pt .
    ?pte cim:IdentifiedObject.name ?pte_name . 
}
""")
df

Tell antall power transformer end pr. power transformer, og sorter synkende på antall.

In [ ]:
df.group_by("pt_name").agg(
    pl.col("pte_name").count().alias("pte_name_count"),
    pl.col("pte_name").alias("pte_name_list")
).sort(
    "pte_name_count", descending=True
)